In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import *
from sklearn.metrics import *

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import gridspec

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [ ]:
model = tf.keras.models.load_model("models/model_MMoE_original")

# Load Data

In [ ]:
import csv

with open('data/test_networks.csv', 'r', newline='') as myfile:
     wr = csv.reader(myfile, quoting=csv.QUOTE_ALL)
     test_data = list(wr)[0]

In [ ]:
df_simulation = pd.read_csv(f'data/data_total.csv', index_col=0)

In [ ]:
df_simulation.columns

In [ ]:
df_simulation_test = df_simulation.loc[df_simulation['id'].isin(test_data)]
df_simulation_test.reset_index(inplace=True)

In [ ]:
performance_measures = ['wait', 'travel', 'timeloss', 'stop', 'speed']

In [ ]:
for pm in performance_measures:
    df_simulation_test.loc[:, pm] = df_simulation_test[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_simulation_test

# Seaborn settings

In [ ]:
sns.set()

In [ ]:
palette = sns.color_palette()

# 1000 simulation sample data for paper

In [ ]:
df_simulation_sample = df_simulation_test.sample(n=1000, random_state=0)

# Original Data EDA

In [ ]:
df_simulation_sample.head()

In [ ]:
df_simulation_sample.columns

In [ ]:
df_simulation_sample

In [ ]:
df_simulation_sample['speed'].describe()

In [ ]:
df_simulation_sample.sort_values(by='wait')[[f'wait{i}' for i in range(5)]].reset_index(drop=True)

## Replication lineplot analysis

In [ ]:
df_simulation_test_pm = dict()

In [ ]:
for pm in performance_measures:
    df_simulation_test_pm[pm] = df_simulation_sample.sort_values(by=pm)[[f'{pm}{i}' for i in range(5)]].reset_index(drop=True)

In [ ]:
sns.set(font_scale=2)

In [ ]:
fig = plt.figure(figsize=(20,8))
axes = []
for i in range(5):
    axes.append(fig.add_subplot(2, 3, i+1))

In [ ]:
pm_title = ['waiting time (s)', 'travel time (s)', 'time loss (s)', 'waiting count (num)', 'speed (m/s)']

In [ ]:
for i, pm in enumerate(performance_measures):
    temp = df_simulation_test_pm[pm].stack().reset_index()
    temp.columns = ['idx', 'pm', pm]
    p = sns.lineplot(data=temp, x="idx", y=pm, ax=axes[i], color=palette[i])
    p.set_ylabel(pm_title[i])
    p.set_xlabel('instance index')

In [ ]:
fig.subplots_adjust(hspace=0.3, wspace=0.3)

In [ ]:
fig

In [ ]:
fig.savefig('figures/original_data_replication.png', bbox_inches='tight')

## Total data box plot

In [ ]:
df_simulation_box_plot = list()
df_simulation_box_plot.append(df_simulation_sample[performance_measures[0:3]].reset_index(drop=True))
df_simulation_box_plot.append(df_simulation_sample[performance_measures[3]].reset_index(drop=True))
df_simulation_box_plot.append(df_simulation_sample[performance_measures[4]].reset_index(drop=True))

In [ ]:
sns.set(font_scale=2.5)

In [ ]:
fig1 = plt.figure(figsize=(20,8))
gs = gridspec.GridSpec(1, 3, width_ratios=[3,1,1])
axes1 = []
for i in range(3):
    axes1.append(plt.subplot(gs[i]))

In [ ]:
p = sns.boxplot(data=df_simulation_box_plot[0], ax=axes1[0], palette=palette)
p.set_xlabel(None)
p.set_xticklabels(['waiting time', 'travel time', 'time loss'])
p.set_ylabel('value (s)')

In [ ]:
p = sns.boxplot(data=df_simulation_box_plot[1], ax=axes1[1], color=palette[3])
p.set_xlabel(None)
p.set_xticklabels(['waiting count'])
p.set_ylabel('value (num)')

In [ ]:
p = sns.boxplot(data=df_simulation_box_plot[2], ax=axes1[2], color=palette[4])
p.set_xlabel(None)
p.set_xticklabels(['speed'])
p.set_ylabel('value (m/s)')

In [ ]:
fig1.subplots_adjust(hspace=0.3, wspace=0.3)

In [ ]:
fig1

In [ ]:
fig1.savefig('figures/original_data_box_plot.png', bbox_inches='tight')

# Model data EDA

In [ ]:
df_comparison = pd.read_csv('data/data_comparison.csv', index_col=0)
df_comparison

In [ ]:
for pm in performance_measures:
    df_comparison.loc[:, f'{pm}_diff'] = df_comparison.loc[:, pm] - df_comparison.loc[:, f'{pm}_pred']

In [ ]:
df_comparison

## Difference box plot

In [ ]:
df_comparison_box_plot = list()
df_comparison_box_plot.append(df_comparison[[f'{pm}_diff' for pm in performance_measures[0:3]]].reset_index(drop=True))
df_comparison_box_plot.append(df_comparison[f'{performance_measures[3]}_diff'].reset_index(drop=True))
df_comparison_box_plot.append(df_comparison[f'{performance_measures[4]}_diff'].reset_index(drop=True))

In [ ]:
sns.set(font_scale=2.7)

In [ ]:
fig2 = plt.figure(figsize=(20,8))
gs = gridspec.GridSpec(1, 3, width_ratios=[3,1,1])
axes2 = []
for i in range(3):
    axes2.append(plt.subplot(gs[i]))

In [ ]:
p = sns.violinplot(data=df_comparison_box_plot[0], ax=axes2[0], palette=palette)
p.set_xlabel(None)
p.set_xticklabels(['waiting time', 'travel time', 'time loss'])
p.set_ylabel('error (s)')
p.set_ylim(-45, 45)

In [ ]:
p = sns.violinplot(data=df_comparison_box_plot[1], ax=axes2[1], color=palette[3])
p.set_xlabel(None)
p.set_xticklabels(['waiting count'])
p.set_ylabel('error (num)')
p.set_ylim(-1.2, 1.2)

In [ ]:
p = sns.violinplot(data=df_comparison_box_plot[2], ax=axes2[2], color=palette[4])
p.set_xlabel(None)
p.set_xticklabels(['speed'])
p.set_ylabel('error (m/s)')
p.set_ylim(-1.2, 1.2)

In [ ]:
fig2.subplots_adjust(wspace=0.4)

In [ ]:
fig2

In [ ]:
fig2.savefig('figures/error_violin_plot.png', bbox_inches='tight')

## MAPE for all test sample intersections

In [ ]:
df_comparison

In [ ]:
for pm in performance_measures:
    df_comparison.loc[:, f'{pm}_mape'] = 100 * abs(df_comparison.loc[:, f'{pm}_diff']) / df_comparison.loc[:, pm]

In [ ]:
df_comparison.head()

In [ ]:
df_comparison.loc[:, 'lanenum'] = df_comparison.loc[:, '1'] + df_comparison.loc[:, '2'] + df_comparison.loc[:, '3'] + df_comparison.loc[:, '4']

In [ ]:
df_comparison_temp = df_comparison.reset_index(drop=True)

In [ ]:
df_mape = df_comparison_temp.groupby('id')[[f'{pm}_mape' for pm in performance_measures]].mean().round(2)
df_mape

In [ ]:
temp = df_comparison_temp.groupby('id')[[f'{pm}_mape' for pm in performance_measures]].std().round(2)
temp.loc[:,:] = temp.loc[:,:]

In [ ]:
df_mape.loc[:,:] = df_mape.loc[:,:].astype(str) + ' (' + temp.loc[:,:].astype(str) + ')'

In [ ]:
df_mape.to_csv('data/test_mape.csv')

## Overall MAPE

In [ ]:
df_comparison[[f'{pm}_mape' for pm in performance_measures]].describe()

In [ ]:
df_comparison[[f'{pm}_diff' for pm in performance_measures]].describe()

## Scatter plot error vs true PM

In [ ]:
sns.set(font_scale=2.5)

In [ ]:
df_tendency_mape = df_comparison.sample(n=1000, random_state=0).reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(20,4))
gs = gridspec.GridSpec(1, 3, width_ratios=[1,1,1])
axes = []
for i in range(3):
    axes.append(plt.subplot(gs[i]))

In [ ]:
temp_df = df_tendency_mape.set_index(['wait'])[['wait_diff']].stack().reset_index()
temp_df['level_1'] = 'waiting time'
temp_df.columns = ['waiting time (s)', 'performance measure', 'value']

p = sns.scatterplot(data=temp_df, x='waiting time (s)', y='value', ax=axes[0], color=palette[0])
p.set_ylabel('error (s)')

In [ ]:
temp_df = df_tendency_mape.set_index(['travel'])[['travel_diff']].stack().reset_index()
temp_df['level_1'] = 'travel time'
temp_df.columns = ['travel time (s)', 'performance measure', 'value']

p = sns.scatterplot(data=temp_df, x='travel time (s)', y='value', ax=axes[1], color=palette[1])
p.set_ylabel('error (s)')

In [ ]:
temp_df = df_tendency_mape.set_index(['timeloss'])[['timeloss_diff']].stack().reset_index()
temp_df['level_1'] = 'time loss'
temp_df.columns = ['time loss (s)', 'performance measure', 'value']

p = sns.scatterplot(data=temp_df, x='time loss (s)', y='value', ax=axes[2], color=palette[2])
p.set_ylabel('error (s)')

In [ ]:
fig.subplots_adjust(wspace=0.35)

In [ ]:
fig

In [ ]:
fig.savefig('figures/error_vs_pm_1.png', bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(20,4))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1])
axes = []
for i in range(2):
    axes.append(plt.subplot(gs[i]))

In [ ]:
temp_df = df_tendency_mape.set_index(['stop'])[['stop_diff']].stack().reset_index()
temp_df['level_1'] = 'waiting count'
temp_df.columns = ['pm', 'performance measure', 'value']

p = sns.scatterplot(data=temp_df, x='pm', y='value', ax=axes[0], color=palette[3])
p.set_xlabel('waiting time (num)')
p.set_ylabel('error (num)')

In [ ]:
temp_df = df_tendency_mape.set_index(['speed'])[['speed_diff']].stack().reset_index()
temp_df['level_1'] = 'speed'
temp_df.columns = ['pm', 'performance measure', 'value']

p = sns.scatterplot(data=temp_df, x='pm', y='value', ax=axes[1], color=palette[4])
p.set_xlabel('speed (m/s)')
p.set_ylabel('error (m/s)')

In [ ]:
fig.subplots_adjust(wspace=0.25)

In [ ]:
fig

In [ ]:
fig.savefig('figures/error_vs_pm_2.png', bbox_inches='tight')

# Tendency performance

In [ ]:
sns.set(font_scale=2.5)

In [ ]:
df_tendency = pd.read_csv('data/tendency_pred.csv', index_col=0)

In [ ]:
df_tendency['wait'] = df_tendency[[f'wait{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_tendency_1 = df_tendency[:94].reset_index(drop=True)
df_tendency_2 = df_tendency[94:187].reset_index(drop=True)
df_tendency_3 = df_tendency[187:280].reset_index(drop=True)
df_tendency_4 = df_tendency[280:].reset_index(drop=True)

In [ ]:
fig = plt.figure(figsize=(20,12))
axes = []
for i in range(4):
    axes.append(fig.add_subplot(2, 2, i+1))

In [ ]:
df_temp = [df_tendency_1, df_tendency_2, df_tendency_3, df_tendency_4]

for i in range(4):
    temp = df_temp[i][[f'p{i+1}', 'wait', 'wait_pred']]
    temp.columns = [f'p{i+1}', 'simulator', 'DNN-based metamodel']
    temp = temp.set_index(f'p{i+1}').stack().reset_index()

    p = sns.lineplot(data=temp, x=f'p{i+1}', y=0, ax=axes[i], hue='level_1')
    
    handles, labels = axes[i].get_legend_handles_labels()
    axes[i].legend(handles=handles[0:], labels=labels[0:])
    
    p.set_ylabel('waiting time (s)')
    p.set_xlabel(f'duration of phase {i+1} (s)')

In [ ]:
fig.subplots_adjust(hspace=0.3, wspace=0.2)

In [ ]:
fig

In [ ]:
fig.savefig('figures/tendency.png', bbox_inches='tight')

# Optimization performance on sample intersection

In [ ]:
import glob

## Tabu

In [ ]:
li = list()
columns = ['p1', 'p2', 'p3', 'p4', 'p1_best', 'p2_best', 'p3_best', 'p4_best', 
    'wait0','travel0', 'stop0', 'timeloss0', 'speed0', 'throughput0',
       'to_be_throughput0', 'wait1', 'travel1', 'stop1', 'timeloss1', 'speed1',
       'throughput1', 'to_be_throughput1', 'wait2', 'travel2', 'stop2',
       'timeloss2', 'speed2', 'throughput2', 'to_be_throughput2', 'wait3',
       'travel3', 'stop3', 'timeloss3', 'speed3', 'throughput3',
       'to_be_throughput3', 'wait4', 'travel4', 'stop4', 'timeloss4', 'speed4',
       'throughput4', 'to_be_throughput4', 'time', 'iteration_time',
       'performance measure', 'veh_generated', 'scenario', 'phaseSeq',
       'algorithm', 'iteration_type', 'step'
]

for filename in glob.glob('data/tabu_optimization/*.csv'):
    temp_df = pd.read_csv(filename, index_col=None, names=columns, header=0)
    temp_df = temp_df.drop(['algorithm', 'iteration_type'], axis=1).astype(float)

    temp_str = filename.split(sep='_')
    temp_df['id'] = '_'.join(temp_str[2:-1])
    temp_df['idx'] = int(temp_str[-1].replace('.csv', ''))
    li.append(temp_df.iloc[-1])

df_tabu = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_tabu = df_tabu.drop([f'p{i}_best' for i in range(1, 5)], axis=1)

In [ ]:
df_tabu = df_tabu[df_tabu.idx < 1250]
df_tabu = df_tabu.reset_index(drop=True)

In [ ]:
df_tabu

In [ ]:
for pm in performance_measures:
    df_tabu.loc[:, pm] = df_tabu[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_tabu.groupby('id').mean()[[*performance_measures, 'iteration_time']]

In [ ]:
df_tabu.groupby('id').mean()[[*performance_measures, 'iteration_time']].to_csv('data/table_tabu.csv')

## ICA

In [ ]:
li = list()
columns = ['p1', 'p2', 'p3', 'p4', 'p1_best', 'p2_best', 'p3_best', 'p4_best', 
    'wait0','travel0', 'stop0', 'timeloss0', 'speed0', 'throughput0',
       'to_be_throughput0', 'wait1', 'travel1', 'stop1', 'timeloss1', 'speed1',
       'throughput1', 'to_be_throughput1', 'wait2', 'travel2', 'stop2',
       'timeloss2', 'speed2', 'throughput2', 'to_be_throughput2', 'wait3',
       'travel3', 'stop3', 'timeloss3', 'speed3', 'throughput3',
       'to_be_throughput3', 'wait4', 'travel4', 'stop4', 'timeloss4', 'speed4',
       'throughput4', 'to_be_throughput4', 'time', 'iteration_time',
       'performance measure', 'veh_generated', 'scenario', 'phaseSeq',
       'algorithm', 'iteration_type', 'step'
]

for filename in glob.glob('data/TLS_ICA/*.csv'):
    temp_df = pd.read_csv(filename, index_col=None, names=columns, header=0)
    temp_df = temp_df[temp_df.iteration_type == 'best']
    temp_df = temp_df.drop(['algorithm', 'iteration_type'], axis=1).astype(float)

    temp_df.iloc[-1, temp_df.columns.get_loc('iteration_time')] = temp_df.iloc[-1].iteration_time + temp_df.iloc[0].iteration_time
    
    temp_str = filename.split(sep='_')
    temp_df['id'] = '_'.join(temp_str[2:-1])
    temp_df['idx'] = int(temp_str[-1].replace('.csv', ''))
    li.append(temp_df.iloc[-1])

df_ica = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_ica = df_ica.drop([f'p{i}_best' for i in range(1, 5)], axis=1)

In [ ]:
df_ica = df_ica[df_tabu.idx < 1250]
df_ica = df_ica.reset_index(drop=True)

In [ ]:
df_ica

In [ ]:
for pm in performance_measures:
    df_ica.loc[:, pm] = df_ica[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_ica.groupby('id').mean()[[*performance_measures, 'iteration_time']]

In [ ]:
df_ica.groupby('id').mean()[[*performance_measures, 'iteration_time']].to_csv('data/table_ica.csv')

## Random traffic signal

In [ ]:
li = list()
columns = ['1', '1l', '1r', '1s', '2', '2l', '2r', '2s', '3', '3l', '3r', '3s',
       '4', '4l', '4r', '4s', '1_2', '1_3', '1_4', '2_1', '2_3', '2_4', '3_1',
       '3_2', '3_4', '4_1', '4_2', '4_3', 'p1','p2', 'p3','p4', 'wait0', 'travel0', 'stop0', 'timeloss0',
       'speed0', 'wait1', 'travel1', 'stop1', 'timeloss1', 'speed1', 'wait2',
       'travel2', 'stop2', 'timeloss2', 'speed2', 'wait3', 'travel3', 'stop3',
       'timeloss3', 'speed3', 'wait4', 'travel4', 'stop4', 'timeloss4',
       'speed4', 'time', 'performance measure', 'id'
]

for filename in glob.glob('data/TLS_random/*.csv'):
    if int(filename.split(sep='_')[-1].replace('.csv', '')) >= 50:
        continue
    
    
    temp_df = pd.read_csv(filename, index_col=0).drop('id', axis=1).mean()
    temp_df['id'] = '_'.join(filename.split(sep='_')[2:-2])
    temp_df.index = columns
    
    li.append(temp_df)

df_random = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_random

In [ ]:
df_random.id.value_counts()

In [ ]:
for pm in performance_measures:
    df_random.loc[:, pm] = df_random[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_random.groupby('id').mean()[[*performance_measures, 'time']]

In [ ]:
df_random.groupby('id').mean()[[*performance_measures, 'time']].to_csv('data/table_random.csv')

## DNN_G3

In [ ]:
li = list()

for filename in glob.glob('data/TLS_DNN_G3/*.csv'):
    if int(filename.split(sep='_')[-1].replace('.csv', '')) >= 50:
        continue
    
    
    temp_df = pd.read_csv(filename, index_col=0)
    temp_id = temp_df.id[0]
    li.append(temp_df.drop([f'{temp_id}_{i}' for i in range(4)], axis=1).loc[0])

df_dnn_g3 = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_dnn_g3

In [ ]:
for pm in performance_measures:
    df_dnn_g3[pm] = df_dnn_g3[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_dnn_g3.groupby('id').mean()[[*performance_measures, 'time']]

In [ ]:
df_dnn_g3.groupby('id').mean()[[*performance_measures, 'time']].to_csv('data/table_DNN_G3.csv')

## DNN_G5

In [ ]:
li = list()

for filename in glob.glob('data/TLS_DNN_G5/*.csv'):
    if int(filename.split(sep='_')[-1].replace('.csv', '')) >= 50:
        continue
    
    
    temp_df = pd.read_csv(filename, index_col=0)
    temp_id = temp_df.id[0]
    li.append(temp_df.drop([f'{temp_id}_{i}' for i in range(4)], axis=1).loc[0])

df_dnn_g5 = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_dnn_g5

In [ ]:
for pm in performance_measures:
    df_dnn_g5[pm] = df_dnn_g5[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_dnn_g5.groupby('id').mean()[[*performance_measures, 'time']]

In [ ]:
df_dnn_g5.groupby('id').mean()[[*performance_measures, 'time']].to_csv('data/table_DNN_G5.csv')

## DNN_G7

In [ ]:
li = list()

for filename in glob.glob('data/TLS_DNN_G7/*.csv'):
    if int(filename.split(sep='_')[-1].replace('.csv', '')) >= 50:
        continue
    
    
    temp_df = pd.read_csv(filename, index_col=0)
    temp_id = temp_df.id[0]
    li.append(temp_df.drop([f'{temp_id}_{i}' for i in range(4)], axis=1).loc[0])

df_dnn_g7 = pd.DataFrame(li).reset_index(drop=True)

In [ ]:
df_dnn_g7

In [ ]:
for pm in performance_measures:
    df_dnn_g7[pm] = df_dnn_g7[[f'{pm}{i}' for i in range(5)]].mean(axis=1)

In [ ]:
df_dnn_g7.groupby('id').mean()[[*performance_measures, 'time']]

In [ ]:
df_dnn_g7.groupby('id').mean()[[*performance_measures, 'time']].to_csv('data/table_DNN_G7.csv')

# Average Waiting Time Comparison for Test Sample Intersections

## Aggregated by ID

In [ ]:
sns.set(font_scale=1.5)

In [ ]:
df_random_agg = df_random.groupby('id').mean()[[*performance_measures, 'time']]
df_tabu_agg = df_tabu.groupby('id').mean()[[*performance_measures, 'iteration_time']]
df_ica_agg = df_ica.groupby('id').mean()[[*performance_measures, 'iteration_time']]
df_dnn_g3_agg = df_dnn_g3.groupby('id').mean()[[*performance_measures, 'time']]
df_dnn_g5_agg = df_dnn_g5.groupby('id').mean()[[*performance_measures, 'time']]
df_dnn_g7_agg = df_dnn_g7.groupby('id').mean()[[*performance_measures, 'time']]

In [ ]:
df_combined = pd.concat([df_random_agg['wait'], df_tabu_agg['wait'], df_ica_agg['wait'], df_dnn_g3_agg['wait'], df_dnn_g5_agg['wait'], df_dnn_g7_agg['wait']], axis=1)
df_combined.columns = ['Random', 'TS', 'ICA+TS', 'DNN_G3', 'DNN_G5', 'DNN_G7']

In [ ]:
# 데이터 melt (long-format)
df_melted = df_combined.melt(var_name='Algorithm', value_name='WaitingTime')

# Random과 나머지를 분리
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [1, 2]})

# 위쪽 (Random만)
sns.boxplot(data=df_melted[df_melted['Algorithm'] == 'Random'], x='Algorithm', y='WaitingTime', ax=ax1, color='grey')
ax1.set_ylim(90, 140)
ax1.spines['bottom'].set_visible(False)
ax1.tick_params(labelbottom=False)
ax1.set_ylabel('')

# 아래쪽 (나머지 알고리즘들)
sns.boxplot(data=df_melted[df_melted['Algorithm'] != 'Random'], x='Algorithm', y='WaitingTime', ax=ax2, color = 'grey')
ax2.set_ylim(10, 40)
ax2.spines['top'].set_visible(False)
ax2.set_ylabel('')
ax2.set_xlabel('')


# 물결 표시
d = .5  # how big to make the diagonal lines
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=10,
              linestyle='none', color='k', mec='k', mew=1, clip_on=False)
ax1.plot([0, 1], [0, 0], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 1], [1, 1], transform=ax2.transAxes, **kwargs)

ax1.axhline(y=90, color='gray', linestyle='--')
ax2.axhline(y=40, color='gray', linestyle='--')

fig.text(0.04, 0.5, 'average waiting time (s)', va='center', rotation='vertical', fontsize=20)


plt.subplots_adjust(hspace=0.05)
plt.savefig('figures/wait_vs_algorithms_boxplot.png', bbox_inches='tight')
plt.show()